pip install ollama-ocr pandas openpyxl langchain_community spacy
python -m spacy download en_core_web_sm


ollama serve


ollama pull llama3.2-vision:11b
ollama pull llama2:13b


Pipeline Medical Prescription

In [ ]:
import os
import pandas as pd
from ollama_ocr import OCRProcessor
from langchain_community.llms import Ollama
import spacy
import re

# Initialize OCR processor with vision model
ocr = OCRProcessor(model_name='llama3.2-vision:11b')

# Initialize Llama 13B text model for NLP
llama = Ollama(base_url='http://localhost:11434', model='llama2:13b')

# Load spaCy for basic NLP preprocessing 
nlp_spacy = spacy.load("en_core_web_sm")

# Directory with prescription images
IMAGE_DIR = r'C:\Users\yayuk\medical prescription images'

# Output files
OCR_OUTPUT_FILE = "ocr_results.xlsx"
STRUCTURED_OUTPUT_FILE = "structured_prescriptions.xlsx"

def extract_text_from_images(image_dir):
    results = []
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, filename)
            print(f"Processing OCR for {filename}...")
            try:
                text = ocr.process_image(image_path, format_type="text")
                results.append({"filename": filename, "extracted_text": text})
            except Exception as e:
                print(f"Error processing {filename}: {repr(e)}")
                results.append({"filename": filename, "extracted_text": ""})
    return pd.DataFrame(results)

def clean_text(text):
    # Basic cleaning - remove excessive whitespace
    return " ".join(text.split())

def extract_medicine_dosage(text):
    """
    Use Llama 13B to extract medicines and dosages in a structured format.
    We prompt Llama to output data in JSON for easier parsing.
    """
    prompt = f"""
You are a medical assistant. Extract the medicines and their dosages from the following prescription text.
Return the result as a JSON list of objects with "medicine" and "dosage" fields.

Prescription text:
\"\"\"
{text}
\"\"\"

JSON output:
"""
    try:
        response = llama.invoke(prompt)
        return response.strip()
    except Exception as e:
        print(f"Error during NLP extraction: {repr(e)}")
        return "[]"

def parse_llama_json_response(response_text):
    import json
    try:
        data = json.loads(response_text)
        if isinstance(data, list):
            return data
        else:
            return []
    except Exception as e:
        print(f"JSON parsing error: {repr(e)}")
        return []

def main():
    # Step 1: OCR extraction
    ocr_df = extract_text_from_images(IMAGE_DIR)
    ocr_df.to_excel(OCR_OUTPUT_FILE, index=False)
    print(f"OCR results saved to {OCR_OUTPUT_FILE}")

    # Step 2: NLP extraction of medicine and dosage
    structured_rows = []
    for idx, row in ocr_df.iterrows():
        filename = row['filename']
        text = clean_text(row['extracted_text'])
        if not text:
            continue

        print(f"Extracting medicines from {filename}...")
        json_response = extract_medicine_dosage(text)
        meds = parse_llama_json_response(json_response)

        if not meds:
            # No meds found, still record empty row for traceability
            structured_rows.append({
                "filename": filename,
                "medicine": None,
                "dosage": None
            })
        else:
            for med in meds:
                structured_rows.append({
                    "filename": filename,
                    "medicine": med.get("medicine"),
                    "dosage": med.get("dosage")
                })

    structured_df = pd.DataFrame(structured_rows)
    structured_df.to_excel(STRUCTURED_OUTPUT_FILE, index=False)
    print(f"Structured prescription data saved to {STRUCTURED_OUTPUT_FILE}")

if __name__ == "__main__":
    main()
